# 1 Intro into torch

## 1.1 Tensors

In [1]:
import torch
import numpy as np

Let's start with the basics. We create some dummy dataset: two observations $n$, every observation has three features $m$. We organize that as a dataset with dimensions $(n,m)$, so that is $(2,3)$

In [2]:
data = [
    [1, 2, 3],
    [10, 20, 30]
]

Our datatype here is `List[int]`, and PyTorch uses a `torch.Tensor` datatype.

In [3]:
X = torch.tensor(data)
type(X)

torch.Tensor

We can retrieve the shape

In [4]:
X.shape

torch.Size([2, 3])

And the type of the data inside the tensor:

In [5]:
X.dtype

torch.int64

Or the amount of observations:

In [6]:
len(X)

2

We can also start with a `numpy.array`

In [7]:
npdata = np.array(
    data,
    dtype = np.float32
)

Note we changed the dataformat to `np.float32`

In [8]:
X2 = torch.from_numpy(npdata)
X2

tensor([[ 1.,  2.,  3.],
        [10., 20., 30.]])

In [9]:
X2.dtype

torch.float32

## 1.2 Usefull functions for creating tensors

We can easily create a stand in tensor, with the same shape as our data:

In [10]:
ones = torch.ones_like(X2)
ones

tensor([[1., 1., 1.],
        [1., 1., 1.]])

Or random weights. These are uniform distributed positive numbers between 0 and 1

In [11]:
X3 = torch.rand(2,3)
X3

tensor([[0.0303, 0.7011, 0.5182],
        [0.2173, 0.1478, 0.9448]])

If we want normally distributed numbers, we need to specify mean and standard deviation:

In [12]:
X4 = torch.normal(mean=0.0, std=0.1, size=(2,3))
X4

tensor([[-0.2964, -0.0248,  0.0289],
        [-0.2669,  0.0431,  0.1096]])

If your laptop or server has a GPU, PyTorch can run the calculations on the GPU. You can check if the GPU can be found by PyTorch with:

In [13]:
torch.cuda.is_available()

False

And you can set the tensor to the GPU device with `.to()`. Default is `"cpu"`

In [14]:
if torch.cuda.is_available():
    tensor = X3.to("cuda")
else:
    print("cuda not found")
X3.device

cuda not found


device(type='cpu')

For people with a macbook with an `mps` backend, there is mps acceleration available.

In [15]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = "cpu"
print(f"Using device {device}")
tensor = X3.to(device)
tensor

Using device cpu


tensor([[0.0303, 0.7011, 0.5182],
        [0.2173, 0.1478, 0.9448]])

Please note that using accelaration with cuda or mps is not always faster!
Reasons why this can be slower are:
- Memory transer: data needs to be transfered from cpu to gpu. This can be a bottleneck.
- parallel processing limits: some architectures (especially the RNNs we will learn about in lesson 3) cant be parallelized. 
- synchronisation overhead: running things in parallel also takes some overhead to synchronise the calculations, like waiting things to finish, merging them back together, etc.

This will especially be true for the simplere models and datasets we are using in the contexts of our lessons.

Other usefull tricks are to create an array of ones. Can you figure out how to create an array of zeros for yourself?

In [16]:
ones = torch.ones(1, 10)
ones

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [17]:
zeros = torch.zeros(1,10)
zeros

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

Tensors can be concatenated. We need to specify the dimension along which the concatenation is done:

In [18]:
torch.cat([ones, ones, ones], dim=0)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

## 1.3 Manipulation of tensors

The basis of most machine learning functions is the linear function. We can easily scale this by using matrix multiplication. Let's say we start with some random data, 32 observations with 10 features.

In [19]:
X = torch.rand(32, 10)

Now, if we want a linear map that transforms these 10 features into 2 dimensions, we can do that with a set of weights with dimensions $(10,2)$

In [20]:
W = torch.rand(10, 2)

In [21]:
yhat = X @ W
yhat.shape

torch.Size([32, 2])

Equivalent is this syntax:

In [22]:
yhat = torch.matmul(X, W)
yhat.shape

torch.Size([32, 2])

Torch will scale this up if you have more dimensions:

In [23]:
X = torch.rand(32, 10, 16)
W = torch.rand(16, 2)
yhat = X @ W
yhat.shape

torch.Size([32, 10, 2])

And finally, we can aggregate the tensor along the two features by taking the mean over the last dimension.

In [24]:
aggregate = yhat.mean(dim=-1)
aggregate.shape

torch.Size([32, 10])

Try for yourself to calculate the sum

In [27]:
sum = yhat.sum()
sum

tensor(2307.6272)

In [28]:
sum_lastdim = yhat.sum(dim=-1)
sum_lastdim

tensor([[ 6.0177,  7.0670,  7.4929,  7.6451,  9.9026,  7.0538,  8.6430,  9.6124,
          7.3747,  7.0149],
        [ 7.9805,  7.2233,  7.1893,  6.3966,  7.1448,  7.1126,  5.8291,  8.4213,
          7.6770,  9.0323],
        [ 8.0446,  7.2922,  5.4883,  9.4742,  5.9974,  8.8418,  7.8549,  5.5187,
          7.5761,  8.9926],
        [ 7.7346,  7.0114,  8.0387,  9.5681,  8.7832,  6.4554,  6.6295,  8.6493,
          8.4422,  7.8146],
        [ 5.7552,  7.7558,  8.1516,  7.0217,  7.3813,  6.7816,  6.3574,  6.4136,
          7.0894,  6.0668],
        [ 6.5010,  7.6402,  7.7193,  6.9347,  4.9714,  8.4914,  7.1966,  5.5150,
          8.7858,  7.5796],
        [ 6.8362,  6.6897,  6.9079,  6.9154,  6.4371,  7.3531,  5.8369,  7.3630,
          7.9716,  4.6259],
        [ 7.3676,  5.8190,  9.9186,  6.2333,  7.5608,  7.3758,  7.0824,  9.5023,
          8.4372,  4.3475],
        [ 7.2862,  7.9430,  5.5412,  5.6838,  6.4173,  8.6021,  7.1840,  6.8863,
          7.6716,  6.3058],
        [ 8.6370,  

In [29]:
sum_lastdim.shape

torch.Size([32, 10])

In [30]:
sum_firstdim = yhat.sum(dim=0)
sum_firstdim.shape

torch.Size([10, 2])

## 1.4 GPU or CPU

Tensors live in the CPU or GPU:

In [31]:
X.device

device(type='cpu')

You can check if you have a GPU available:

In [32]:
torch.cuda.is_available()

False

Or a mac with M1

In [33]:
torch.backends.mps.is_available()

False

And move a tensor to the GPU for faster computing, if available

In [34]:
if torch.cuda.is_available():
    X_ = X.to("cuda")
elif torch.backends.mps.is_available():
    X_ = X.to("mps")
else:
    X_ = X.to("cpu")
X_.device

device(type='cpu')

## 1.5 Reshape or View

Often, you will need to reshape a tensor:

In [35]:
X = torch.rand(32, 28, 28, 1)
X_view = X.view(32, 28*28)
X_reshape = X.reshape(32, 28*28)
X.shape, X_view.shape, X_reshape.shape

(torch.Size([32, 28, 28, 1]), torch.Size([32, 784]), torch.Size([32, 784]))

The difference between `view` and `reshape` is: `view` operates as a view on the original tensor. If the underlying data is changed, the view will change too.

No data movement occurs when creating a view, view tensor just changes the way it interprets the same data.

In [36]:
X = torch.Tensor([0, 0])
X_view = X.view(1,2)
X.storage().data_ptr() == X_view.storage().data_ptr()

C:\Users\dilek\AppData\Local\Temp\ipykernel_46320\3787215844.py:3: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  X.storage().data_ptr() == X_view.storage().data_ptr()


True

In [37]:
X[0] = 1
X_view

tensor([[1., 0.]])

`view` can throw an error if the required view is not contiguous (does not share the same memory block)

> A tensor whose values are laid out in the storage starting from the rightmost dimension onward (that is, moving along rows for a 2D tensor) is defined as contiguous. Contiguous tensors are convenient because we can visit them efficiently in order without jumping around in the storage (improving data locality improves performance because of the way memory access works on modern CPUs). This advantage of course depends on the way algorithms visit.

You could call `.contiugous()` on a `view`, but `.reshape()` does that behind the scenes.

## 1.6 Permute

Sometimes you might want to reshuffle the order of a tensor.

For example, let's say we load an batch of 32 images, where every image has a size of 28x28 pixels, and has 3 channels (RGB color)

In [38]:
X = torch.rand(32, 28, 28, 3)

It is the case that there are different conventions for manipulating tensors in image recognition models. Some models have a channel-last convention, like I used above, but some (like [pytorch](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)) use a channel first convention, which would be (batch, channel, height, width).

You would want to swap the 4th dimension to the 2nd, or if you start from zero:

In [39]:
channel_first = X.permute(0, 3, 1, 2)
channel_first.shape

torch.Size([32, 3, 28, 28])

## 1.7 Broadcasting

Broadcasting is something you might know from `numpy`, but it is also used by `tensorflow`, `jax` and `torch`. 

Broadcasting allows to extend a dimension, without the need to do so explicitly. The rules for broadcasting are simple:

- two dimesions are equal
- one of the dimensions is 1

but lets show an example

In [40]:
a = torch.ones(2, 2)
b = torch.ones(2, 2)
a, b, a+b

(tensor([[1., 1.],
         [1., 1.]]),
 tensor([[1., 1.],
         [1., 1.]]),
 tensor([[2., 2.],
         [2., 2.]]))

This is straigh forward. But what would happen in this case:

In [41]:
a = torch.ones(1, 2)
b = torch.ones(2, 2)

`b` is a 2x2 grid, and has four numbers. If we want to add `a`, we have only two numbers! Now, you could start stacking the `a` tensor to get matching dimensions. But you dont have to!

In [42]:
a + b

tensor([[2., 2.],
        [2., 2.]])

See what happened here? 

`a` is magically broadcasted over the first dimension. And what would you guess would happen in this case:

In [47]:
a = torch.ones(1, 5, 1, 4)
b = torch.ones(3, 1, 3, 1)
x = a+b
x.shape

torch.Size([3, 5, 3, 4])

First, predict the output shape, then check it for yourself.

And, what would you think happens here; do you think this gives an error, or do you think it broadcasts?

In [48]:
a = torch.ones(5, 1, 4)
b = torch.ones(3, 1, 3, 1)
y = a+b 
y.shape

torch.Size([3, 5, 3, 4])